# How do I make my pandas DataFrame smaller and faster?
https://www.youtube.com/watch?v=wDYDYGyN_cw

In [1]:
import pandas as pd

In [2]:
drinks = pd.read_csv("data/drinks.csv")

In [3]:
#df in genel bilgisi verir.
drinks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
country                         193 non-null object
beer_servings                   193 non-null int64
spirit_servings                 193 non-null int64
wine_servings                   193 non-null int64
total_litres_of_pure_alcohol    193 non-null float64
continent                       193 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 9.1+ KB


In [6]:
# objectlerin gerçek alanlarını da verir.
drinks.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
country                         193 non-null object
beer_servings                   193 non-null int64
spirit_servings                 193 non-null int64
wine_servings                   193 non-null int64
total_litres_of_pure_alcohol    193 non-null float64
continent                       193 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 25.9 KB


In [7]:
# 1. sine göre her bir column usage
drinks.memory_usage()

Index                             72
country                         1544
beer_servings                   1544
spirit_servings                 1544
wine_servings                   1544
total_litres_of_pure_alcohol    1544
continent                       1544
dtype: int64

In [8]:
# 2. sine göre her bir column usage
drinks.memory_usage(deep=True)

Index                              72
country                         10272
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                       10016
dtype: int64

In [9]:
# 2. sine göre toplam
drinks.memory_usage(deep=True).sum()

26536

In [10]:
# yukarıdaki referanslarla unique continent değerlerini buluyoruz.
sorted(drinks.continent.unique())

['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']

In [11]:
# her birini ds olarak listelersek bir unique id oluşacak
drinks.continent.head()

0      Asia
1    Europe
2    Africa
3    Europe
4    Africa
Name: continent, dtype: object

In [12]:
#buradan bir lookup table oluşturabiliriz. çünkü lookup table bize yer kazandıracaktır. 
#kıtaların adlarını her kayıtta tekrar etmek zorunda kalmayacağız. db normalization, foreign key gibi
drinks['continent'] = drinks.continent.astype("category")

In [16]:
#categorynin değiştiğini görürüz.
drinks.dtypes

country                           object
beer_servings                      int64
spirit_servings                    int64
wine_servings                      int64
total_litres_of_pure_alcohol     float64
continent                       category
dtype: object

In [18]:
# hala tabloyu çağırırken görünen değişmeyecek, aynı şeyi göreceğiz.
#fakat aşağıdaki dtype kısmında categories diye birşey görürüz. aslında integer olarak tutulmaktadır.
drinks.continent.head()

0      Asia
1    Europe
2    Africa
3    Europe
4    Africa
Name: continent, dtype: category
Categories (6, object): [Africa, Asia, Europe, North America, Oceania, South America]

In [19]:
# buradan integer olduğunu görebiliriz.
drinks.continent.cat.codes.head()

0    1
1    2
2    0
3    2
4    0
dtype: int8

In [20]:
#bunun memory_usage a etkisi 
drinks.memory_usage(deep=True)

Index                              72
country                         10272
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                         672
dtype: int64

In [21]:
# bonus logical order
df = pd.DataFrame({'ID': [100,101,102,103], 'quality':['good','very good','good','excellent']})

In [22]:
df

,ID,quality
0,100,good
1,101,very good
2,102,good
3,103,excellent


In [23]:
# bu df i quality alanına göre sıralarsak alfabetik bir sıra olduğunu görürüz ama bizim ihtiyacımız bu değilse?
df.sort_values('quality')

,ID,quality
3,103,excellent
0,100,good
2,102,good
1,101,very good


In [26]:
# logical category oluşturabiliriz.
df['quality'] = df.quality.astype('category', categories=['good','very good', 'excellent'],ordered=True)

In [28]:
# df e bakınca farklı birşey yok, fakat allta category ve büyüklük tanımlarını görürürüz.
df.quality

0         good
1    very good
2         good
3    excellent
Name: quality, dtype: category
Categories (3, object): [good < very good < excellent]

In [31]:
# sıraladığımızda artık logical order a  göre sıralandığını görürüz
df.sort_values('quality')

,ID,quality
0,100,good
2,102,good
1,101,very good
3,103,excellent


In [32]:
# hatta sıralama comparison bile yapabiliriz.
df.loc[df.quality > 'good']

,ID,quality
1,101,very good
3,103,excellent
